## V5

In [ ]:
# PR.usp_AOMS_ApplicationWiseContinuousData
#1 Missed Subquery detection, detected everything else:
# WHERE AL.Metric_PA_Key NOT IN 
# (
# SELECT DISTINCT Metric_PA_Key 
# FROM [PR].[AOMS_DecommissionedMetricsList] (NOLOCK)
# WHERE IsActive = 1
# )


#2 

In [2]:
prompt = f"""
You are a SQL data lineage extractor specializing in T-SQL stored procedures.

OBJECTIVE:
Extract DIRECT source-to-target mappings between PERSISTENT database objects ONLY.
Trace data flow through ALL intermediate steps (temp tables, CTEs, subqueries) but report only the FINAL persistent objects.

OBJECT CLASSIFICATION:

PERSISTENT OBJECTS (Report these):
- Tables: schema.table, [schema].[table], database.schema.table
- Views: schema.view, [schema].[view]
- Stored procedures (when used as data sources via EXEC INSERT)

INTERMEDIATE OBJECTS (Trace through, but DO NOT report):
- Temp tables: #temp, ##global_temp
- Table variables: @table
- CTEs: WITH cte_name AS (...)
- Subqueries and derived tables
- Variables: @variable

EXTRACTION RULES:

1. TRACE THROUGH INTERMEDIATES:
   - If temp table #T is populated from table A, then #T is inserted into table B
   - Report: A → B (not A → #T or #T → B)

2. HANDLE MULTI-STEP FLOWS:
   - Step 1: A → #temp1
   - Step 2: #temp1 → #temp2  
   - Step 3: #temp2 → B
   - Report: A → B

3. MULTIPLE SOURCES TO ONE TARGET:
   - Create separate lineage entries for each source
   - Example: A → C, B → C (two separate JSON objects)

4. ONE SOURCE TO MULTIPLE TARGETS:
   - Create separate lineage entries for each target
   - Example: A → X, A → Y (two separate JSON objects)

5. COMPLEX QUERIES:
   - Trace through all JOINs, subqueries, CTEs
   - Extract base tables from nested SELECT statements
   - Follow data flow through UNION, EXCEPT, INTERSECT operations

6. IGNORE:
   - Table hints: (NOLOCK), WITH (NOLOCK), (INDEX=...), etc.
   - System tables/views unless explicitly part of business logic
   - The stored procedure name itself as a source

7. DELETE/TRUNCATE OPERATIONS:
   - These affect targets but have no sources
   - Omit from lineage (or include with "source": null if you need to track modifications)

8. EXEC STORED PROCEDURES:
   - If "INSERT INTO table EXEC stored_proc", treat stored_proc as a source
   - Otherwise, you may need to trace into that procedure separately

OUTPUT FORMAT:

{{
  "lineage": [
    {{
      "source": "schema.table_name",
      "target": "schema.table_name"
    }}
  ]
}}

RULES ENFORCEMENT:
✓ Output ONLY valid JSON
✓ No explanations, comments, or markdown
✓ No temp tables (#temp) in final output
✓ No CTEs or table variables in final output
✓ Each lineage pair must have exactly one source and one target
✓ Use fully qualified names when available (schema.table)
✓ Remove all table hints from object names

EXAMPLE:

Given SQL:
```sql
-- Step 1: Read from A, B into temp
SELECT * INTO #temp FROM A JOIN B ON A.id = B.id

-- Step 2: Read from #temp and C into final table
INSERT INTO Z 
SELECT * FROM #temp JOIN C ON #temp.id = C.id
```

Correct output:
{{
  "lineage": [
    {{"source": "A", "target": "Z"}},
    {{"source": "B", "target": "Z"}},
    {{"source": "C", "target": "Z"}}
  ]
}}

Incorrect output (DO NOT DO THIS):
{{
  "lineage": [
    {{"source": "A", "target": "#temp"}},
    {{"source": "B", "target": "#temp"}},
    {{"source": "#temp", "target": "Z"}},
    {{"source": "C", "target": "Z"}}
  ]
}}

SQL TO ANALYZE:
{sql_text}
"""

NameError: name 'sql_text' is not defined

## V4

In [ ]:
prompt= f"""
You are a Data Engineering specialist. Your task is to generate a FLATTENED data lineage.

### THE GOLDEN RULE
- ONLY permanent tables (e.g., [Schema].[Table]) are allowed in the output.
- TEMPORARY TABLES (starting with '#') ARE FORBIDDEN.
- If data goes TableA -> #Temp -> TableB, you MUST record it as TableA -> TableB.

### EXTRACTION LOGIC
1. **Identify the Final Target:** Find the last `INSERT INTO` or `UPDATE` affecting a permanent table.
2. **Trace Backwards:** Look at every table that contributed data to that final target, even if it passed through multiple temp tables (#temp3a, #temp3b, etc.).
3. **Handle Joins:** If the data was formed by joining Table A and Table B, both are individual sources for the final target.
4. **Ignore Filter-only Tables:** Do not include tables found only in `WHERE` subqueries (like lookup tables for IDs) unless they provide actual columns to the final target.

### STRICT OUTPUT FORMAT
- Return ONLY valid JSON.
- No explanation. No comments.
- Format:
{{
  "lineage": [
    {{ "source": "Permanent_Source_1", "target": "Permanent_Target" }},
    {{ "source": "Permanent_Source_2", "target": "Permanent_Target" }}
  ]
}}

### PROHIBITED TERMS
If the following appear in your "source" or "target" fields, the output is WRONG:
- Any name starting with '#'
- Any name starting with '@'
- Any name containing 'CTE'

### EXAMPLE OF FLATTENING
SQL: 
SELECT * INTO #T1 FROM SourceA;
SELECT * INTO #T2 FROM #T1 JOIN SourceB ON ...;
INSERT INTO TargetFinal SELECT * FROM #T2;

Desired JSON:
{{
  "lineage": [
    {{ "source": "SourceA", "target": "TargetFinal" }},
    {{ "source": "SourceB", "target": "TargetFinal" }}
  ]
}}

### SQL TO ANALYZE:
{sql_text}
"""

## V3

In [ ]:
prompt= f"""
You are a SQL Data Lineage Architect. Your goal is to map the absolute end-to-end flow of data between permanent tables, bypassing all intermediate logic.

### TASK
Analyze the provided SQL and extract EVERY relationship between original permanent Source objects and the final permanent Target objects.

### STRICT RULES (MANDATORY)
1. **COMPREHENSIVE SOURCE DISCOVERY:**
   - Scan the entire script including `FROM` clauses, `JOIN` clauses, and subqueries.
   - If multiple tables are joined to create a result that eventually lands in a target, EVERY table used in those joins is a separate "source" for that "target".

2. **LINEAGE COLLAPSING (NO TEMP TABLES):**
   - Temporary tables (e.g., `#temp3a`, `#temp3b`) must NEVER appear in the output.
   - If the flow is `[TableA] + [TableB] -> #Temp -> [FinalTable]`, your output must be:
     - {{ "source": "[TableA]", "target": "[FinalTable]" }}
     - {{ "source": "[TableB]", "target": "[FinalTable]" }}

3. **OUTPUT FORMAT:**
   - Return ONLY a valid JSON object: {{"lineage": [ {{"source": "...", "target": "..."}} ]}}
   - No explanations, no markdown.

4. **OBJECT CLEANING:**
   - Strip all SQL hints like `(NOLOCK)`.
   - Do NOT include variables (`@RM`) or the Stored Procedure name itself.

### EXAMPLE
Input: 
INSERT INTO FinalTable SELECT * FROM TableA JOIN TableB ON ...
Output:
{{
  "lineage": [
    {{ "source": "TableA", "target": "FinalTable" }},
    {{ "source": "TableB", "target": "FinalTable" }}
  ]
}}

### SQL TO ANALYZE:
{sql_text}
"""

## V2

In [ ]:
custom_prompt = f"""
You are a SQL Data Lineage Architect. Your goal is to map the end-to-end flow of data between permanent tables, ignoring all intermediate steps.

### TASK
Analyze the provided SQL and extract ONLY the relationship between the original permanent Source objects and the final permanent Target objects.

### STRICT RULES (MANDATORY)
1.  **LINEAGE COLLAPSING (CRITICAL):** - Temporary tables (starting with '#') are internal logic and must NOT appear in the output.
    - If data flows `PermanentSource -> #TempTable` and then `#TempTable -> PermanentTarget`, you must output: `PermanentSource -> PermanentTarget`.
    - Effectively "bridge" the gap created by temp tables to show the true origin and destination.

2.  **OUTPUT FORMAT:**
    - Return ONLY a valid JSON object.
    - No markdown, no explanations, no comments.
    - Structure: {{"lineage": [{{"source": "...", "target": "..."}}]}}

3.  **OBJECT IDENTIFICATION:**
    - Only include real objects (schema.table, schema.view, database.schema.table).
    - Strip all SQL hints like `(NOLOCK)` or `WITH (NOLOCK)`.
    - Do NOT treat the Stored Procedure name itself as a source or target.

4.  **NEGATIVE CONSTRAINTS (DO NOT INCLUDE):**
    - DO NOT include any object starting with '#'.
    - DO NOT include variables (starting with '@').
    - DO NOT include CTEs (Common Table Expressions) as sources or targets; link their underlying tables to the final target.

### EXAMPLE
Input SQL: 
SELECT * INTO #Buffer FROM [Sales].[Orders];
INSERT INTO [Archive].[Orders_History] SELECT * FROM #Buffer;

Output JSON:
{{
  "lineage": [
    {{ "source": "[Sales].[Orders]", "target": "[Archive].[Orders_History]" }}
  ]
}}

### SQL TO ANALYZE:
{sql_text}
"""

## V1

In [ ]:
prompt= f"""
You are a SQL data lineage extractor.

TASK:
Extract ALL source-to-target data object mappings from the SQL.

STRICT RULES (MANDATORY):
- Output ONLY valid JSON
- No explanations
- No comments
- No markdown

OBJECT IDENTIFICATION RULES:
- A valid source or target must be a real data object:
  - schema.table
  - database.schema.table
  - [schema].[table]

- DO NOT include SQL table hints:
  - IGNORE and REMOVE: NOLOCK, (NOLOCK), WITH (NOLOCK)

- DO NOT treat SQL keywords or hints as schemas


PAIRING RULES:
- Each source MUST be paired with exactly one target
- DO NOT group sources
- DO NOT group targets
- One JSON object per source → target relationship
- If multiple sources write to the same target, repeat the target

TEMP TABLE RULES:
- Temp tables (#table) are INTERMEDIATE objects
- DO NOT use temp tables as final targets
- If a temp table feeds a permanent table, map source → permanent table
- Use temp tables ONLY if no permanent target exists


STORED PROCEDURE RULES:
- Do NOT treat stored procedure names as source tables
- Extract underlying base tables used inside the procedure
- Final lineage must represent table-to-table movement


REQUIRED OUTPUT FORMAT:

{{
  "lineage": [
    {{
      "source": "string",
      "target": "string"
    }}
  ]
}}

EXAMPLE:
If SQL reads from A, B (WITH NOLOCK) and inserts into C,
output MUST be:
{{
  "lineage": [
    {{ "source": "A", "target": "C" }},
    {{ "source": "B", "target": "C" }}
  ]
}}

SQL:
{sql_text}
"""